In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os

TRAIN_DIR = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip"
TEST_DIR = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip"

df = pd.read_csv(TRAIN_DIR)
df = df.drop("id", axis=1)
comments = df.comment_text.values
rating = df.loc[:, "toxic":].values

In [ ]:
print(len(rating[0]))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(comments)

In [ ]:
encoded_comments = tokenizer.texts_to_sequences(comments)

In [ ]:
max_input_length = max(list(map(len, encoded_comments)))
input_data = np.array(pad_sequences(encoded_comments, maxlen=max_input_length, padding="post"))
num_words = len(tokenizer.word_index) + 1

In [ ]:
from keras import Input, Model
from keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
 
inputs = Input(shape=(None,), dtype="int64")

x = Embedding(num_words, 128)(inputs)
x = Dropout(0.5)(x)
x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)

predictions = Dense(6, activation="sigmoid", name="predictions")(x)

model = Model(inputs, predictions)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
print(input_data.shape, rating.shape)

In [ ]:
epochs = 3
model.fit(input_data, rating, epochs=epochs)

In [ ]:
def str_to_tokens(sentence):
    tokens = []
    for word in sentence.lower().split():
        try:
            tokens.append(tokenizer.word_index[word])
        except KeyError:
            tokens.append(0)
    return pad_sequences([tokens], maxlen=max_input_length, padding="post")

In [ ]:
string = comments[0]
translated = str_to_tokens(string)
prediction = model.predict(translated)[0]
print(prediction)
np.argsort(prediction)[::-1]

In [ ]:
model.save("model.h5")
import pickle
with open("tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)